In [1]:
import pandas as pd
import re
import requests 
import pyodbc
from bs4 import BeautifulSoup
from string import ascii_uppercase
import re
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
productionDB = True
year = 2022
seasonId = 5
currentWeek = 3

connection = ''
conn1 = ''
conn2 = ''

if productionDB:
    connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
    conn1 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
    conn2 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
else:
    connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')
    conn1 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')
    conn2 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')


In [3]:
def GetPlayerId(playerName):
    cursor = conn1.cursor()
    cursor.execute("SELECT Id FROM Players WHERE [ProReferenceKey] = ?", playerName)
    playerId = 0
    for row in cursor.fetchall():
        playerId = row[0]
        return playerId

In [4]:
def GetTeamId(team):
    return {
        'ARI':1,
        'CRD':1,
        'ATL':2,
        'BAL':3,
        'RAV':3,
        'BUF':4,
        'CAR':5,
        'CHI':6,
        'CIN':7,
        'CLE':8,
        'DAL':9,
        'DEN':10,
        'DET':11,
        'GNB':12,
        'HOU':13,
        'HTX':13,
        'IND':14,
        'CLT':14,
        'JAX':15,
        'KAN':16,
        'LVR':17,
        'RAI':17,
        'LAC':18,
        'SDG':18,
        'LAR':19,
        'RAM':19,
        'MIA':20,
        'MIN':21,
        'NWE':22,
        'NOR':23,
        'NYG':24,
        'NYJ':25,
        'PHI':26,
        'PIT':27,
        'SFO':28,
        'SEA':29,
        'TAM':30,
        'TEN':31,
        'OTI':31,
        'WAS':32,
        '':'',
        '2TM':''
    }[team]

In [5]:
cursor = conn1.cursor()


cursor.execute("SELECT Id, CurrentWeek FROM Seasons WHERE [Current] = 1")
for row in cursor.fetchall():
    seasonId = row[0]
    currentWeek = row[1]
        
cursor.close()
print(seasonId)
print(currentWeek)

5
3


In [6]:
# OFFENSIVE STATS

cursor = conn1.cursor()


cursor.execute("SELECT BoxscoreLink FROM NFLSchedule WHERE [SeasonId] = ? AND [Week] = ?",seasonId, currentWeek)
for row in cursor.fetchall():
    link = row[0]
    print(link)
    
    url = 'https://www.pro-football-reference.com{}'.format(link)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    offensiveTable = soup.find("table", id="player_offense")
    if offensiveTable is not None:
        for player in offensiveTable.find_all('tbody')[0].find_all('tr'):
            playerKey = ''

            if len(player.findAll('th')[0].findAll('a', href=True)) > 0:
                playerKey = player.findAll('th')[0].findAll('a', href=True)[0]['href'].split('/')[3].split('.')[0]        

            if playerKey != '':
                playerId = GetPlayerId(playerKey)
                pass_att = 0
                if len(player.findAll("td", {"data-stat" : "pass_att"})) > 0:
                    pass_att = player.findAll("td", {"data-stat" : "pass_att"})[0].get_text()

                pass_cmp = 0
                if len(player.findAll("td", {"data-stat" : "pass_cmp"})) > 0:
                    pass_cmp = player.findAll("td", {"data-stat" : "pass_cmp"})[0].get_text()

                pass_yds = 0
                if len(player.findAll("td", {"data-stat" : "pass_yds"})) > 0:
                    pass_yds = player.findAll("td", {"data-stat" : "pass_yds"})[0].get_text()

                pass_td = 0
                if len(player.findAll("td", {"data-stat" : "pass_td"})) > 0:
                    pass_td = player.findAll("td", {"data-stat" : "pass_td"})[0].get_text()

                pass_int = 0
                if len(player.findAll("td", {"data-stat" : "pass_int"})) > 0:
                    pass_int = player.findAll("td", {"data-stat" : "pass_int"})[0].get_text()

                rush_yds = 0
                if len(player.findAll("td", {"data-stat" : "rush_yds"})) > 0:
                    rush_yds = player.findAll("td", {"data-stat" : "rush_yds"})[0].get_text()

                rush_att = 0
                if len(player.findAll("td", {"data-stat" : "rush_att"})) > 0:
                    rush_att = player.findAll("td", {"data-stat" : "rush_att"})[0].get_text()

                rush_td = 0
                if len(player.findAll("td", {"data-stat" : "rush_td"})) > 0:
                    rush_td = player.findAll("td", {"data-stat" : "rush_td"})[0].get_text()

                targets = 0
                if len(player.findAll("td", {"data-stat" : "targets"})) > 0:
                    targets = player.findAll("td", {"data-stat" : "targets"})[0].get_text()


                rec = 0
                if len(player.findAll("td", {"data-stat" : "rec"})) > 0:
                    rec = player.findAll("td", {"data-stat" : "rec"})[0].get_text()

                rec_yds = 0
                if len(player.findAll("td", {"data-stat" : "rec_yds"})) > 0:
                    rec_yds = player.findAll("td", {"data-stat" : "rec_yds"})[0].get_text()

                rec_td = 0
                if len(player.findAll("td", {"data-stat" : "rec_td"})) > 0:
                    rec_td = player.findAll("td", {"data-stat" : "rec_td"})[0].get_text()

                fumbles_lost = 0
                if len(player.findAll("td", {"data-stat" : "fumbles_lost"})) > 0:
                    fumbles_lost = player.findAll("td", {"data-stat" : "fumbles_lost"})[0].get_text()

                if playerId is None:
                    print('No Player found for {}'.format(playerKey))
                else:

                    sqlq = "SELECT COUNT(1) FROM WeeklyPlayerStats WHERE NFLWeek = {} AND PlayerId = {} and SeasonId = {}".format(currentWeek, playerId, seasonId)
                    cursor.execute(sqlq)
                    if cursor.fetchone()[0]:
                        cursor.execute("UPDATE WeeklyPlayerStats SET [PassYards] = ?, [PassAttempts] = ?, [Completions] = ?,[PassTDs] = ?,[Interceptions] = ?,[RushYards] = ?,[RushTDs] = ?,[RushAttempts] = ?,[Fumbles] = ?,[Receptions] = ?,[Targets] = ?,[RecYards] = ?,[RecTDs] = ? WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?", pass_yds,pass_att, pass_cmp, pass_td,pass_int,rush_yds,rush_td,rush_att,fumbles_lost,rec,targets,rec_yds,rec_td,currentWeek, playerId, seasonId,)
                        cursor.commit()
                        print("Update {} Passing: {}/{} {} yards {} TD {} int Rushing: {} attempts for {} yards and {} TDs  Passing: {} targets with {} comp for {} yards and {} TDs".format(playerId, pass_cmp, pass_att, pass_yds, pass_td, pass_int, rush_att, rush_yds, rush_td, targets, rec, rec_yds, rec_td ))
                    else:
                        cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [PassYards], [PassAttempts],[Completions],[PassTDs],[Interceptions],[RushYards],[RushTDs],[RushAttempts],[Fumbles],[Receptions],[Targets],[RecYards],[RecTDs]) Values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",currentWeek, playerId, seasonId, pass_yds,pass_att, pass_cmp, pass_td,pass_int,rush_yds,rush_td,rush_att,fumbles_lost,rec,targets,rec_yds,rec_td)
                        cursor.commit()
                        print("Add {} Passing: {}/{} {} yards {} TD {} int Rushing: {} attempts for {} yards and {} TDs  Passing: {} targets with {} comp for {} yards and {} TDs".format(playerId, pass_cmp, pass_att, pass_yds, pass_td, pass_int, rush_att, rush_yds, rush_td, targets, rec, rec_yds, rec_td ))

cursor.close()


/boxscores/202209220cle.htm
Update 1358 Passing: 20/32 207 yards 0 TD 0 int Rushing: 2 attempts for 7 yards and 1 TDs  Passing: 0 targets with 0 comp for 0 yards and 0 TDs
Update 1533 Passing: 0/0 0 yards 0 TD 0 int Rushing: 15 attempts for 56 yards and 1 TDs  Passing: 3 targets with 3 comp for 5 yards and 0 TDs
Update 1732 Passing: 0/0 0 yards 0 TD 0 int Rushing: 4 attempts for 30 yards and 0 TDs  Passing: 1 targets with 1 comp for 3 yards and 0 TDs
Update 250 Passing: 0/0 0 yards 0 TD 0 int Rushing: 1 attempts for 11 yards and 0 TDs  Passing: 6 targets with 3 comp for 35 yards and 0 TDs
Update 732 Passing: 0/0 0 yards 0 TD 0 int Rushing: 0 attempts for 0 yards and 0 TDs  Passing: 11 targets with 8 comp for 84 yards and 0 TDs
Update 1539 Passing: 0/0 0 yards 0 TD 0 int Rushing: 0 attempts for 0 yards and 0 TDs  Passing: 4 targets with 2 comp for 41 yards and 0 TDs
Update 1617 Passing: 0/0 0 yards 0 TD 0 int Rushing: 0 attempts for 0 yards and 0 TDs  Passing: 7 targets with 3 comp for 

In [7]:
cursor = conn1.cursor()


cursor.execute("SELECT BoxscoreLink FROM NFLSchedule WHERE [SeasonId] = ? AND [Week] = ?",seasonId, currentWeek)
for row in cursor.fetchall():
    link = row[0]
    print(link)
    
    url = 'https://www.pro-football-reference.com{}'.format(link)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    scoringTable = soup.find("table", id="scoring")
    playerIdsScored = []
    if scoringTable is not None:
        for play in scoringTable.find_all('tbody')[0].find_all('tr'):
            scoringPlay = play.findAll('td')[2]
            if "field goal" in scoringPlay.get_text():
                playerKey = ''

                if len(scoringPlay.findAll('a', href=True)) > 0:
                    playerName = scoringPlay.findAll('a', href=True)[0].get_text()
                    playerKey = scoringPlay.findAll('a', href=True)[0]['href'].split('/')[3].split('.')[0]        

                play = scoringPlay.get_text()
                distance = int(play[len(playerName)+1:len(play)].split(' ')[0])
                
                if playerKey != '':
                    playerId = GetPlayerId(playerKey)
                    if playerId is None:
                        print('No Player found for {}'.format(playerKey))
                    else:
                        if playerId not in playerIdsScored:
                            deleteStatement = "DELETE FROM WeeklyPlayerStats WHERE NFLWeek = {} AND PlayerId = {} and SeasonId = {}".format(currentWeek, playerId, seasonId)
                            cursor.execute(deleteStatement)
                            playerIdsScored.append(playerId)
                        sqlq = "SELECT COUNT(1) FROM WeeklyPlayerStats WHERE NFLWeek = {} AND PlayerId = {} and SeasonId = {}".format(currentWeek, playerId, seasonId)
                        cursor.execute(sqlq)
                        if cursor.fetchone()[0]:
                            if distance < 20:
                                cursor.execute("UPDATE WeeklyPlayerStats SET  [FieldGoals1to19Made] = [FieldGoals1to19Made] + 1 WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",currentWeek, playerId, seasonId)
                            if distance >= 20 and distance < 30:
                                cursor.execute("UPDATE WeeklyPlayerStats SET [FieldGoals20to29Made] = [FieldGoals20to29Made] + 1 WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",currentWeek, playerId, seasonId)
                            if distance >= 30 and distance < 40:
                                cursor.execute("UPDATE WeeklyPlayerStats SET [FieldGoals20to29Made] = [FieldGoals30to39Made] + 1 WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",currentWeek, playerId, seasonId)
                            if distance >= 40 and distance < 50:
                                cursor.execute("UPDATE WeeklyPlayerStats SET [FieldGoals40to49Made] = [FieldGoals40to49Made] + 1 WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",currentWeek, playerId, seasonId)
                            if distance >= 50:
                                cursor.execute("UPDATE WeeklyPlayerStats SET [FieldGoals50PlusMade] = [FieldGoals50PlusMade] + 1 WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",currentWeek, playerId, seasonId)
                            cursor.commit()
                            print("Update {} ({}) - {} yd FG".format(playerName, playerId, distance))
                        else:
                            if distance < 20:
                                cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [FieldGoals1to19Made] ) Values (?,?,?,?)",currentWeek, playerId, seasonId, 1)
                            if distance >= 20 and distance < 30:
                                cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [FieldGoals20to29Made] ) Values (?,?,?,?)",currentWeek, playerId, seasonId, 1)
                            if distance >= 30 and distance < 40:
                                cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [FieldGoals30to39Made] ) Values (?,?,?,?)",currentWeek, playerId, seasonId, 1)
                            if distance >= 40 and distance < 50:
                                cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [FieldGoals40to49Made] ) Values (?,?,?,?)",currentWeek, playerId, seasonId, 1)
                            if distance >= 50:
                                cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [FieldGoals50PlusMade] ) Values (?,?,?,?)",currentWeek, playerId, seasonId, 1)
                            cursor.commit()
                            print("Add {} ({}) - {} yd FG".format(playerName, playerId, distance))

                            
    kickingTable = soup.find("table", id="kicking")
    if kickingTable is not None:
        for kicker in kickingTable.find_all('tbody')[0].find_all('tr'):
            print(kicker)
cursor.close()


/boxscores/202209220cle.htm
Add Cade York (1640) - 34 yd FG
Add Chris Boswell (122) - 34 yd FG
/boxscores/202209250car.htm
Add Eddy Pineiro (1067) - 45 yd FG
Update Eddy Pineiro (1067) - 48 yd FG
Update Eddy Pineiro (1067) - 34 yd FG
/boxscores/202209250chi.htm
Add Cairo Santos (1173) - 47 yd FG
Update Cairo Santos (1173) - 50 yd FG
Add Ka'imi Fairbairn (408) - 39 yd FG
Update Ka'imi Fairbairn (408) - 23 yd FG
Update Cairo Santos (1173) - 30 yd FG
/boxscores/202209250clt.htm
Add Chase McLaughlin (909) - 43 yd FG
No Player found for AmmeMa00
Update Chase McLaughlin (909) - 51 yd FG
/boxscores/202209250mia.htm
Add Tyler Bass (74) - 30 yd FG
/boxscores/202209250min.htm
Add Austin Seibert (1194) - 40 yd FG
/boxscores/202209250nwe.htm
Add Nick Folk (436) - 35 yd FG
Update Nick Folk (436) - 50 yd FG
Add Justin Tucker (1359) - 56 yd FG
/boxscores/202209250nyj.htm
Add Greg Zuerlein (1507) - 50 yd FG
Update Greg Zuerlein (1507) - 40 yd FG
Add Evan McPherson (1569) - 22 yd FG
Update Evan McPhers

In [8]:
cursor = conn1.cursor()


cursor.execute("SELECT BoxscoreLink FROM NFLSchedule WHERE [SeasonId] = ? AND [Week] = ?",seasonId, currentWeek)
for row in cursor.fetchall():
    link = row[0]
    print(link)

    url = 'https://www.pro-football-reference.com{}'.format(link)

    DRIVER_PATH = 'C:/ChromeDriver/chromedriver.exe'
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

    driver.implicitly_wait(4)
    kickTable = driver.find_elements(By.ID, 'kicking')
    if len(kickTable) == 0:
        driver.close()
        continue
    rows = kickTable[0].find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME,'tr')
    for i in rows:
        try:
            playerKey = i.find_element(By.TAG_NAME, 'a').get_attribute('href').split('/')[5].split('.')[0]
            playerId = 0
            if playerKey != '':
                playerId = GetPlayerId(playerKey)
            if playerId is not None:
                xpm = i.find_elements(By.TAG_NAME, 'td')[1].text
                if len(xpm) > 0:
                    sqlq = "SELECT COUNT(1) FROM WeeklyPlayerStats WHERE NFLWeek = {} AND PlayerId = {} and SeasonId = {}".format(currentWeek, playerId, seasonId)
                    cursor.execute(sqlq)
                    if cursor.fetchone()[0]:
                        cursor.execute("UPDATE WeeklyPlayerStats SET [PATMade] = ? WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",xpm,currentWeek, playerId, seasonId)
                        cursor.commit()
                        print("UPDATE {} ({}) made {} XP".format(playerKey,playerId,xpm))
                    else:
                        cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [PATMade] ) Values (?,?,?,?)",currentWeek, playerId, seasonId, xpm)
                        cursor.commit()
                        print("INSERT {} made {} XP".format(playerKey,xpm))
        except NoSuchElementException:
            continue

    returnTable = driver.find_elements(By.ID, 'returns')
    rows = returnTable[0].find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME,'tr')
    for i in rows:
        try:
            playerKey = i.find_element(By.TAG_NAME, 'a').get_attribute('href').split('/')[5].split('.')[0]
            playerId = 0
            if playerKey != '':
                playerId = GetPlayerId(playerKey)
            krYards = i.find_elements(By.TAG_NAME, 'td')[2].text
            krTD = i.find_elements(By.TAG_NAME, 'td')[4].text
            prYards = i.find_elements(By.TAG_NAME, 'td')[7].text
            prTD = i.find_elements(By.TAG_NAME, 'td')[9].text
            if playerId is not None:
                sqlq = "SELECT COUNT(1) FROM WeeklyPlayerStats WHERE NFLWeek = {} AND PlayerId = {} and SeasonId = {}".format(currentWeek, playerId, seasonId)
                cursor.execute(sqlq)
                if cursor.fetchone()[0]:
                    cursor.execute("UPDATE WeeklyPlayerStats SET [KRYards] = ?, [KRTDs] = ?, [PRYards] = ?, [PRTDs] = ? WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",krYards,krTD,prYards,prTD,currentWeek, playerId, seasonId)
                    cursor.commit()
                    print("UPDATE {} ({}) KR Yards: {} TD: {}  PR Yards: {} TD:{}".format(playerKey,playerId,krYards,krTD,prYards,prTD))
                else:
                    cursor.execute("INSERT INTO WeeklyPlayerStats ([NFLWeek],[PlayerId], [SeasonId], [KRYards],[KRTDs],[PRYards],[PRTDs] ) Values (?,?,?,?,?,?,?)",currentWeek, playerId, seasonId, krYards,krTD,prYards,prTD)
                    cursor.commit()
                    print("INSERT {} ({}) KR Yards: {} TD: {}  PR Yards: {} TD:{}".format(playerKey,playerId,krYards,krTD,prYards,prTD))

        except NoSuchElementException:
            continue
    driver.close()
cursor.close()


/boxscores/202209220cle.htm


[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 9.62MB/s]
C:\Users\Micha\AppData\Local\Temp\ipykernel_17436\583753973.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


UPDATE BoswCh00 (122) made 2 XP
UPDATE YorkCa00 (1640) made 2 XP
UPDATE OlszGu00 (1010) KR Yards: 18 TD: 0  PR Yards: 0 TD:0
UPDATE RogeCh02 (1140) KR Yards: 0 TD: 0  PR Yards: 11 TD:0
UPDATE BellDa02 (1632) KR Yards: 12 TD: 0  PR Yards: 0 TD:0
UPDATE FordJe00 (1656) KR Yards: 15 TD: 0  PR Yards: 0 TD:0
/boxscores/202209250car.htm
INSERT LutzWi00 made 2 XP
UPDATE PineEd00 (1067) made 1 XP
UPDATE LandJa00 (809) KR Yards: 0 TD: 0  PR Yards: 17 TD:0
INSERT WashDw00 (1397) KR Yards: 56 TD: 0  PR Yards: 0 TD:0
UPDATE SmitSh03 (1582) KR Yards: 0 TD: 0  PR Yards: 22 TD:0
UPDATE ShenLa00 (1199) KR Yards: 54 TD: 0  PR Yards: 0 TD:0
/boxscores/202209250chi.htm
UPDATE FairKa01 (408) made 2 XP
UPDATE SantCa01 (1173) made 2 XP
INSERT PettDa00 (1060) KR Yards: 0 TD: 0  PR Yards: 1 TD:0
UPDATE EbneTr00 (1670) KR Yards: 52 TD: 0  PR Yards: 0 TD:0
/boxscores/202209250clt.htm
UPDATE McLaCh00 (909) made 2 XP
INSERT MoorSk01 (1619) KR Yards: 0 TD: 0  PR Yards: 12 TD:0
UPDATE PachIs00 (1680) KR Yards: 94 T

In [9]:
cursor = conn1.cursor()


cursor.execute("SELECT BoxscoreLink FROM NFLSchedule WHERE [SeasonId] = ? AND [Week] = ?",seasonId, currentWeek)
for row in cursor.fetchall():
    link = row[0]
    print(link)
    url = 'https://www.pro-football-reference.com{}'.format(link)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    

    scoreContainer = soup.find("table", {"class": "linescore"})
    if scoreContainer is not None:
        rows = scoreContainer.find_all('tbody')[0].find_all('tr')
        homeScore = int(rows[1].find_all('td')[-1].get_text())
        awayScore = int(rows[0].find_all('td')[-1].get_text())
        
        DRIVER_PATH = 'C:/ChromeDriver/chromedriver.exe'
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)

        driver.implicitly_wait(4)
        teamStatTable = driver.find_elements(By.ID, 'team_stats')
        
        headerRow = teamStatTable[0].find_elements(By.TAG_NAME, 'thead')[0].find_elements(By.TAG_NAME,'tr')[0]
        awayTeam = headerRow.find_elements(By.TAG_NAME, 'th')[1].text
        homeTeam = headerRow.find_elements(By.TAG_NAME, 'th')[2].text
        
        totalYardRow = teamStatTable[0].find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME,'tr')[5]
        awayYardsAllowed = totalYardRow.find_elements(By.TAG_NAME, 'td')[1].text
        homeYardsAllowed = totalYardRow.find_elements(By.TAG_NAME, 'td')[0].text
        
        offensiveTable = driver.find_elements(By.ID, 'player_offense')
        offensiveRows = offensiveTable[0].find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME,'tr')

        homeFR = 0
        awayFR = 0
        
        if offensiveTable is not None:
            for oRow in offensiveRows:
                cols = oRow.find_elements(By.TAG_NAME, 'td')
                if len(cols) > 0:
                    team = cols[0].text
                    if team == homeTeam:
                        awayFR = awayFR + int(cols[20].text)
                    if team == awayTeam:
                        homeFR = homeFR + int(cols[20].text)

        defenseTable = driver.find_elements(By.ID, 'player_defense')
        defenseRows = defenseTable[0].find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME,'tr')
        homeInt = 0
        awayInt = 0
        
        homeTd = 0
        awayTd = 0
        
        homeSack = 0
        awaySack = 0
        for row in defenseRows:
            columns = row.find_elements(By.TAG_NAME, 'td')
            if len(columns) > 0:
                team = columns[0].text
                if team == homeTeam:
                    homeInt = homeInt + int(columns[1].text)
                    homeTd = homeTd + int(columns[3].text)
                    homeSack = homeSack + float(columns[6].text)
                if team == awayTeam:
                    awayInt = awayInt + int(columns[1].text)
                    awayTd = awayTd + int(columns[3].text)
                    awaySack = awaySack + float(columns[6].text)
        print("Points Allowed - Home: {} vs Away: {}".format(awayScore - homeTd*6,homeScore - awayTd *6))
        print("Yards - Home ({}): {} vs Away ({}): {}".format(homeTeam,homeYardsAllowed,awayTeam,awayYardsAllowed))
        print("Int - Home: {} vs Away: {}".format(homeInt,awayInt))
        print("TD - Home: {} vs Away: {}".format(homeTd,awayTd))
        print("Sack - Home: {} vs Away: {}".format(homeSack,awaySack))
        print("Fumble Recovery - Home: {} vs Away: {}".format(homeFR,awayFR))

        sqlq = "SELECT COUNT(1) FROM WeeklyPlayerStats WHERE NFLWeek = {} AND PlayerId = {} and SeasonId = {}".format(currentWeek, GetTeamId(homeTeam) * -1, seasonId)
        cursor.execute(sqlq)
        if cursor.fetchone()[0]:
            cursor.execute("UPDATE WeeklyPlayerStats SET [PointsAllowed] = ?, [YardsAllowed] = ?, [Sacks] = ?, [DefensiveTD] = ?, [DefensiveInt] = ?, [DefensiveFumbleRecovery] = ? WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",awayScore - homeTd*6,homeYardsAllowed,homeSack,homeTd,homeInt,homeFR,currentWeek, GetTeamId(homeTeam) * -1, seasonId)
            cursor.commit()
            print("UPDATE")
        else:
            cursor.execute("INSERT INTO WeeklyPlayerStats ([PointsAllowed],[YardsAllowed],[Sacks],[DefensiveTD],[DefensiveInt],[DefensiveFumbleRecovery],[NFLWeek],[PlayerId], [SeasonId]) VALUES(?,?,?,?,?,?,?,?,?)",awayScore - homeTd*6,homeYardsAllowed,homeSack,homeTd,homeInt,homeFR,currentWeek, GetTeamId(homeTeam) * -1, seasonId)
            cursor.commit()
            print("INSERT")
            
        sqlq = "SELECT COUNT(1) FROM WeeklyPlayerStats WHERE NFLWeek = {} AND PlayerId = {} and SeasonId = {}".format(currentWeek, GetTeamId(awayTeam) * -1, seasonId)
        cursor.execute(sqlq)
        if cursor.fetchone()[0]:
            cursor.execute("UPDATE WeeklyPlayerStats SET [PointsAllowed] = ?, [YardsAllowed] = ?, [Sacks] = ?, [DefensiveTD] = ?, [DefensiveInt] = ?, [DefensiveFumbleRecovery] = ? WHERE [NFLWeek] = ? AND [PlayerId] = ? AND [SeasonId] = ?",homeScore - awayTd*6,awayYardsAllowed,awaySack,awayTd,awayInt,awayFR,currentWeek, GetTeamId(awayTeam) * -1, seasonId)
            cursor.commit()
            print("UPDATE")
        else:
            cursor.execute("INSERT INTO WeeklyPlayerStats ([PointsAllowed],[YardsAllowed],[Sacks],[DefensiveTD],[DefensiveInt],[DefensiveFumbleRecovery],[NFLWeek],[PlayerId], [SeasonId]) VALUES(?,?,?,?,?,?,?,?,?)",homeScore - awayTd*6,awayYardsAllowed,awaySack,awayTd,awayInt,awayFR,currentWeek, GetTeamId(awayTeam) * -1, seasonId)
            cursor.commit()
            print("INSERT")

        driver.close()
cursor.close()

/boxscores/202209220cle.htm


C:\Users\Micha\AppData\Local\Temp\ipykernel_17436\4264288302.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Points Allowed - Home: 17 vs Away: 29
Yards - Home (CLE): 308 vs Away (PIT): 376
Int - Home: 0 vs Away: 0
TD - Home: 0 vs Away: 0
Sack - Home: 1.0 vs Away: 2.0
Fumble Recovery - Home: 1 vs Away: 0
UPDATE
UPDATE
/boxscores/202209250car.htm
Points Allowed - Home: 14 vs Away: 22
Yards - Home (CAR): 426 vs Away (NOR): 293
Int - Home: 2 vs Away: 0
TD - Home: 0 vs Away: 0
Sack - Home: 1.0 vs Away: 3.0
Fumble Recovery - Home: 1 vs Away: 0
INSERT
INSERT
/boxscores/202209250chi.htm
Points Allowed - Home: 20 vs Away: 23
Yards - Home (CHI): 329 vs Away (HOU): 363
Int - Home: 2 vs Away: 2
TD - Home: 0 vs Away: 0
Sack - Home: 1.0 vs Away: 5.0
Fumble Recovery - Home: 0 vs Away: 0
INSERT
INSERT
/boxscores/202209250clt.htm
Points Allowed - Home: 17 vs Away: 20
Yards - Home (IND): 315 vs Away (KAN): 259
Int - Home: 1 vs Away: 0
TD - Home: 0 vs Away: 0
Sack - Home: 1.0 vs Away: 5.0
Fumble Recovery - Home: 0 vs Away: 1
INSERT
INSERT
/boxscores/202209250mia.htm
Points Allowed - Home: 19 vs Away: 21
Yards 